<a href="https://colab.research.google.com/github/lorransr/codas-method/blob/main/CODAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Método **CO**mbinative **D**istance-based **AS**sessment (CODAS)


[artigo](https://ideas.repec.org/a/cys/ecocyb/v50y2016i3p25-44.html)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#mostrando 3 casas decimais no array do numpy
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

This example is adapted from Chakraborty and Zavadskas (2014) which is
related to the selection of the most appropriate industrial robot. Five different
criteria which are considered in this robot selection problem are: load capacity (in
kg), maximum tip speed (in mm/s), repeatability (in mm), memory capacity (in
points or steps) and manipulator reach (in mm). Among these criteria, the load
capacity, maximum tip speed, memory capacity, and manipulator reach are defined
as benefit criteria, and the repeatability is defined as a cost criterion


Exemplo de seleção de um robo industrial apropriado (Chakraborty e Zavadskas, 2014).

Cinco Critérios considerados:
* Carga Suportada (Kg) (L [<sup>1</sup>](#fn1))
* Velocidade maxima na ponta (mm/s) (L)
* repetibilidade (mm) (C [<sup>2</sup>](#fn2))
* capacidade de memória (em pontos ou passos) (L)

<span id="fn1">1: L = Lucro </span>
<span id="fn2">2: C = Custo </span>


In [ ]:
# inputs
data = {
    "load_capacity": [60,6.35,6.8,10,2.5,4.5,3],
    "repeatability" : [0.4,0.15,0.10,0.2,0.10,0.08,0.1],
    "maximum_tip_speed" : [2540,1016,1727.2,1000,560,1016,1778],
    "memory_capacity" : [500,3000,1500,2000,500,350,1000],
    "manipulator_reach" : [990,1041,1676,965,915,508,920]
    }

w = [0.036, 0.192, 0.326, 0.326, 0.120]
benefit_criterias = ["load_capacity","maximum_tip_speed","memory_capacity","manipulator_reach"]
cost_criterias = ["repeatability"]

## Passo 1 
- Construir a matriz de decisão (X), da forma:

$$ X = [x_{ij}]_{n \times m} = \begin{vmatrix} 
   x_{11} & x_{12} & ... & x_{1m}  \\
   x_{21} & x_{22} & ... &  x_{2m}  \\
  : & : &: &  :  \\
    x_{n1} & x_{n2} & ... &  x_{nm}  \\
   \end{vmatrix}  $$


onde $x_{ij}$ ($x_{ij} >= 0$) denota o valor de performance da i-ésima alternativa no j-ésimo cretério


In [ ]:
m_raw = pd.DataFrame(data)
alternatives = [ "a_" + str(i) for i in range(1,len(m_raw)+1)]
m_raw.index = alternatives
m_raw

,load_capacity,repeatability,maximum_tip_speed,memory_capacity,manipulator_reach
a_1,60.00,0.40,2540.0,500,990
a_2,6.35,0.15,1016.0,3000,1041
a_3,6.80,0.10,1727.2,1500,1676
a_4,10.00,0.20,1000.0,2000,965
a_5,2.50,0.10,560.0,500,915
a_6,4.50,0.08,1016.0,350,508
a_7,3.00,0.10,1778.0,1000,920


In [ ]:

weights = pd.Series(w,index=m_raw.columns)
weights

load_capacity        0.036
repeatability        0.192
maximum_tip_speed    0.326
memory_capacity      0.326
manipulator_reach    0.120
dtype: float64

## Passo 2
- calcular a matriz de decisão normalizada. Deve ser usada a normalização linear do valor da performance de acordo com a formula:

$$n_{ij} =
\begin{cases}
    \frac{x_{ij}}{\max x_{ij}} & \text{se } j \in N_{b}\\
    \frac{\min x_{ij}}{x_{ij}},& \text{se } j \in N_{c} 
\end{cases}
$$


onde $N_b$ e $N_c$ representam os conjuntos de lucro e custo, respectivamente.


In [ ]:
def normalize_codas(s:pd.Series,benefit_criteria = True):
  if benefit_criteria:
    return s.apply(lambda x: x/(s.max()))
  else:
    return s.apply(lambda x: s.min()/x)


In [ ]:
m_raw[["maximum_tip_speed"]].apply(lambda x: normalize_codas(x,benefit_criteria=True))

,maximum_tip_speed
a_1,1.000000
a_2,0.400000
a_3,0.680000
a_4,0.393701
a_5,0.220472
a_6,0.400000
a_7,0.700000


In [ ]:
benefits = m_raw[benefit_criterias].apply(lambda x: normalize_codas(x))
costs = m_raw[cost_criterias].apply(lambda x: normalize_codas(x,benefit_criteria=False))

m_normalized = pd.concat([benefits,costs],axis=1)
m_normalized

,load_capacity,maximum_tip_speed,memory_capacity,manipulator_reach,repeatability
a_1,1.000000,1.000000,0.166667,0.590692,0.200000
a_2,0.105833,0.400000,1.000000,0.621122,0.533333
a_3,0.113333,0.680000,0.500000,1.000000,0.800000
a_4,0.166667,0.393701,0.666667,0.575776,0.400000
a_5,0.041667,0.220472,0.166667,0.545943,0.800000
a_6,0.075000,0.400000,0.116667,0.303103,1.000000
a_7,0.050000,0.700000,0.333333,0.548926,0.800000


## Passo 3
Calcular a matriz normalizada e ponderada de acordo com a equação:

$$r_{ij} = w_{j}n_{ij}$$

onde $w_j$ ($0 < w_j < 1$) representa o j-ésimo critério e 

$$\sum^m_{j=1}w_j = 1$$

In [ ]:
assert 1 == (weights).sum(), "somatório dos pesos deve ser 1"

In [ ]:
m_weighted = m_normalized * weights
m_weighted

,load_capacity,manipulator_reach,maximum_tip_speed,memory_capacity,repeatability
a_1,0.03600,0.070883,0.326000,0.054333,0.0384
a_2,0.00381,0.074535,0.130400,0.326000,0.1024
a_3,0.00408,0.120000,0.221680,0.163000,0.1536
a_4,0.00600,0.069093,0.128346,0.217333,0.0768
a_5,0.00150,0.065513,0.071874,0.054333,0.1536
a_6,0.00270,0.036372,0.130400,0.038033,0.1920
a_7,0.00180,0.065871,0.228200,0.108667,0.1536


## Passo 4
- Determinar a solução ideal negativa
da seguinte forma:

$$ns = [ns_j]_{1 \times m}$$

$$ns_j = \min r_{ij}$$

In [ ]:
negative_ideal_solution = m_weighted.min()
negative_ideal_solution

load_capacity        0.001500
manipulator_reach    0.036372
maximum_tip_speed    0.071874
memory_capacity      0.038033
repeatability        0.038400
dtype: float64

## Passo 5 
- Calcular a distância Euclidiana e a distância Manhattan  das alternativas em relação a **solução ideal negativa**, como mostrado a seguir:

$$E_i = \sqrt{\sum^m_{j=1}(r_{ij}-ns_j)^2}$$

$$T_i = \sum^m_{j=1}|r_{ij}-ns_j|$$

In [ ]:
euclidian_distance = m_weighted.apply(lambda x: np.linalg.norm(x - negative_ideal_solution),axis=1)
euclidian_distance

a_1    0.259282
a_2    0.303163
a_3    0.241516
a_4    0.194687
a_5    0.119941
a_6    0.164377
a_7    0.208730
dtype: float64

In [ ]:
manhathan_distance = m_weighted.apply(lambda x: (x - negative_ideal_solution).abs().sum(),axis=1)
manhathan_distance

a_1    0.339437
a_2    0.450965
a_3    0.476180
a_4    0.311393
a_5    0.160641
a_6    0.213326
a_7    0.371958
dtype: float64

## Passo 6
- Construir a matriz de determinação

$$Ra = [h_{ik}]_{n \times m}$$

$$h_{ik} = (E_i - E_k) + (\psi(E_i - E_k) \times (T_i - T_k))$$

onde k $\in \{1,2,...,n\}$ e $\psi$ é a função limite que reconhece a similaridade das distâncias euclidianas de duas alternativas, tal função é definida como:

$$\psi(x) 
\begin{cases} 
0, & \text{se } |x|\ge \tau \\
1, & \text{se } |x|\lt \tau
\end{cases}$$

na função, $\tau$ é um parâmetro que pode ser escolhido pelo decisor. Levando-se em consideração que é recomendado que seja proximo de 0; de forma que, caso a diferença entre as distâncias euclidianas das alternativas for maior que $\tau$ essas duas alternativas também devem ser comparadas em termos da distância Manhattan.

Para esse exemplo $\tau = 0.02$ 

In [ ]:
threshold = 0.02

#matriz comparação
def comparison_matrix(s:pd.Series)->np.array:
  return s.values - s.values[:,None]

euclidean_comparison = pd.DataFrame(
    comparison_matrix(euclidian_distance),
    index=alternatives,
    columns=alternatives
    )

manhathan_comparison = pd.DataFrame(
    comparison_matrix(manhathan_distance),
    index=alternatives,
    columns=alternatives)

print("euclidean")
print(euclidean_comparison)
print("--------------------------"*3)
print("manhattan")
print(manhathan_comparison)

euclidean
          a_1       a_2       a_3       a_4       a_5       a_6       a_7
a_1  0.000000  0.043881 -0.017766 -0.064594 -0.139340 -0.094905 -0.050552
a_2 -0.043881  0.000000 -0.061647 -0.108476 -0.183222 -0.138786 -0.094433
a_3  0.017766  0.061647  0.000000 -0.046828 -0.121574 -0.077139 -0.032786
a_4  0.064594  0.108476  0.046828  0.000000 -0.074746 -0.030310  0.014043
a_5  0.139340  0.183222  0.121574  0.074746  0.000000  0.044435  0.088789
a_6  0.094905  0.138786  0.077139  0.030310 -0.044435  0.000000  0.044353
a_7  0.050552  0.094433  0.032786 -0.014043 -0.088789 -0.044353  0.000000
------------------------------------------------------------------------------
manhattan
          a_1       a_2       a_3       a_4       a_5       a_6       a_7
a_1  0.000000  0.111528  0.136744 -0.028044 -0.178796 -0.126111  0.032521
a_2 -0.111528  0.000000  0.025215 -0.139572 -0.290324 -0.237639 -0.079007
a_3 -0.136744 -0.025215  0.000000 -0.164787 -0.315540 -0.262854 -0.104222
a_4  0.028044

In [ ]:
((euclidean_comparison.abs()<threshold)*1)

,a_1,a_2,a_3,a_4,a_5,a_6,a_7
a_1,1,0,1,0,0,0,0
a_2,0,1,0,0,0,0,0
a_3,1,0,1,0,0,0,0
a_4,0,0,0,1,0,0,1
a_5,0,0,0,0,1,0,0
a_6,0,0,0,0,0,1,0
a_7,0,0,0,1,0,0,1


In [ ]:
threshold_matrix = ((euclidean_comparison.abs()<threshold)*1)
relative_assesment_matrix = (
    euclidean_comparison + threshold_matrix  * manhathan_comparison)

relative_assesment_matrix

,a_1,a_2,a_3,a_4,a_5,a_6,a_7
a_1,0.000000,0.043881,0.118978,-0.064594,-0.139340,-0.094905,-0.050552
a_2,-0.043881,0.000000,-0.061647,-0.108476,-0.183222,-0.138786,-0.094433
a_3,-0.118978,0.061647,0.000000,-0.046828,-0.121574,-0.077139,-0.032786
a_4,0.064594,0.108476,0.046828,0.000000,-0.074746,-0.030310,0.074608
a_5,0.139340,0.183222,0.121574,0.074746,0.000000,0.044435,0.088789
a_6,0.094905,0.138786,0.077139,0.030310,-0.044435,0.000000,0.044353
a_7,0.050552,0.094433,0.032786,-0.074608,-0.088789,-0.044353,0.000000


In [ ]:
assessment_score = relative_assesment_matrix.sum().sort_values(ascending=False)
assessment_score

a_2    0.630446
a_3    0.335658
a_1    0.186532
a_7    0.029979
a_4   -0.189450
a_6   -0.341059
a_5   -0.652106
dtype: float64